# Libraries import

In [21]:
import requests
from pathlib import Path
from tqdm.auto import tqdm
from tqdm.utils import CallbackIOWrapper
import zipfile

# Functions definition

In [19]:
def check_bulk_upload_file(filename):
    if Path(filename).suffix != ".zip":
        raise Exception("Bulk_upload file should be a zipped file")
    with zipfile.ZipFile(bulk_upload_file) as z:
        if not any("manifest" in x.lower() for x in z.namelist()):
            Exception("Manifest file missing from the zip file")

In [1]:
def bulk_upload_creation(filename):
    url = f"{DOMAIN}/service/genome-files/bulk-upload/"
    
    payload = {"filename": filename}
    
    response = requests.post(url,
                             json=payload,
                             headers=headers)
    response.raise_for_status()
    print("Bulk_upload_id",response.json()['id'],"created successfully.")
    return response.json()["id"], response.json()["url"]

In [5]:
def upload_file(file_path, upload_url):
    file_size = Path(file_path).stat().st_size
    
    with open(file_path, "rb") as f:
        with tqdm(total=file_size, unit="B", unit_scale=True, unit_divisor=1024, desc=file_path) as t:
            wrapped_file = CallbackIOWrapper(t.update, f, "read")
            response = requests.put(upload_url,
                                    data=wrapped_file)
            response.raise_for_status()

In [6]:
def finish_bulk_upload(upload_id):
    url = f"{DOMAIN}/service/genome-files/bulk-upload/{upload_id}/"
    
    response = requests.delete(url,
                               headers=headers)
    response.raise_for_status()
    print(f"Bulk_upload_id {upload_id} finished successfully.")

In [7]:
def bulk_upload_status(upload_id):
    url = f"{DOMAIN}/service/genome-files/bulk-upload/{upload_id}/"
    
    response = requests.get(url,
                               headers=headers)
    response.raise_for_status()
    return response.json()

In [ ]:
def bulk_upload(filename):
    check_bulk_upload_file(filename)
    upload_id, upload_url = bulk_upload_creation(filename)
    upload_file(filename, upload_url)
    finish_bulk_upload(upload_id)
    return upload_id

# Variables setup

In [3]:
DOMAIN = "https://qa-selfdecode.com"
auth_token = "Token ###"

headers = {"Accept": "application/json",
           "Content-Type": "application/json",
           "Authorization": auth_token}

In [7]:
bulk_upload_file = 'GI_batch1.zip'

_Notes:_
- The file to be uploaded must be a zipped file (.zip)
- The zipped file must include the GTC files and a _manifest_ file
- The _manifest_ file must:
  - Have "manifest" in its name
  - Be comma or white-space delimited (.csv or .txt)
  - Have column names in first row
  - have _'sample_id'_, _'call_rate'_, and _'filename'_ columns, where _'sample_id'_ would be the barcode

# Commands

## Upload bulk file

In [ ]:
upload_id = bulk_upload(bulk_upload_file)

## Check upload status

In [ ]:
bulk_upload_status(upload_id)